# Programming Model

Structured Streaming 은 SparkSQL 위에서 동작하며, sql 의 최적화 기법이나 falut-tolerance, scalability 등을 모두 상속받음

Structured Streaming 의 핵심 아이디어는, 데이터 계속해서 append 되는 테이블(```Unbounded Input Table```)을 통해 데이터 스트림을 처리하는 것

![Unbounded Tabel](https://spark.apache.org/docs/3.0.3/img/structured-streaming-stream-as-a-table.png)

Unbounded Input Table 위에서 동작하는 쿼리로써, 배치 프로세싱 모델과 똑같은 코드로 스트림 처리를 가능하게 함

정적 테이블에 대한 연산으로 표현하고, 이를 내부적으로 스트림 잡으로 변환하여 실행

즉, 배치 데이터를 테이블로 받아오고, 테이블을 조작하거나 집계를 내는 등의 다양한 SQL 쿼리를 통해 데이터를 처리할 수 있다면, 이를 스트림처리에 그대로 가져와서 스트림 데이터가 계속 들어오는 Unbounded Table 에 적용할 수 있음

DataFrame/DataSet api 를 사용하여 테이블에 대해 Aggregation, Window, Join 등이 가능

테이블에 행한 쿼리는 결과 테이블(Result Table)을 만들어내는데, 사용자가 지정한 trigger interval 마다 새로운 데이터들이 테이블에 추가되면서 Result Table 을 갱신

![trigger](https://spark.apache.org/docs/3.0.3/img/structured-streaming-model.png)

소스로부터 데이터를 읽어들이고, 테이블을 처리하여 결과테이블을 업데이트한 뒤 소스를 버림 - 결과테이블 갱신에 필요한 중간 데이터는 최소한으로 보관

많은 다른 스트리밍 시스템들은 사용자가 falut-tolerance 나 data-consistency 를 보증하기 위해서 처리하거나 데이터를 유지해야할 부분이 많지만, Structured Streaming 은 결과테이블 갱신만 책임지면 된다는 점에서 부담감이 적음

## 요약

1. 데이터가 ```소스```로부터 정의된 스키마에 맞게 들어옴
2. 이벤트 스트림이란 ```unbounded table``` 에 추가되는 rows
3. 스트림으로부터 결과를 얻기 위해 unbounded table 에서 쿼리를 날림
4. 동일한 쿼리를 테이블에 trigger 마다 반복적으로 날려 이벤트의 처리 ```결과 테이블```을 생성
5. 결과를 ```싱크```에 제공

---

## 스파크 세션

스파크쉘에서 돌릴 경우는 SparkContext 처럼 SparkSession 이 제공됨

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [ ]:
spark = SparkSession.builder.appName("StructuredStreamingTest").getOrCreate()

---

## 소스 (Source)
소스란? 스트리밍 데이터 공급자를 나타내는 추상화된 개념 == 단순히 데이터가 생성되는 곳

구조적 스트리밍은 외부 스트리밍 소스에서 현재 오프셋을 요청하고, 자기가 이전에 마지막으로 처리한 오프셋과 비교하여 그 사이에 있는 배치 데이터를 가져와 처리.
- 신뢰할 수 있는 소스란? 스트리밍 소스를 동일한 순서로 다시 재생시킬 수 있어야 함(스트리밍 프로세스가 실패하더라도, 커밋되지 않은 오프셋을 재생성해서 다시 처리할 수 있어야 함) == ```replayability```

In [ ]:
# value 라는 column 하나, 스트리밍하게 입력받는 데이터들이 각각 row 를 구성하는 DataFrame
# Warning 은 무시. 소켓으로부터 스트림을 읽는 것은 서비스 배포용이 아닌 학습용으로만 사용하라는 경고
lines = spark.readStream.format("socket").option("host", "localhost").option("port", "5000").load()
print(lines)

### format
- file : 파일시스템에 존재하는 파일을 읽어옴, 배치 기반 프로세스를 스트리밍 시스템으로 연결하기 위한 간단한 방법. 스파크 2.3.0 부터 csv, json, parquet, orc, text, textfile 등을 지원
- kafka : 카프카 토픽으로부터 데이터를 컨슘해오는 subscriber
- **socket : utf-8 로 인코딩된 텍스트 데이터 스트림을 제공하는 tcp 서버에 연결해서 데이터를 받아오는 tcp 클라이언트**
- rate : 초당 n개의 레코드를 생성해내게끔 설정해줄 수 있는 스트림 생성기

### load

load 를 통해 받아온 결과는 스트리밍되는 DataFrame

|   | value |
| --- | --- |
| 1 | Hello World |
| 2 | Word Count |
| 3 | Spark Streaming |
| 4 | Streaming World |
| 5 | Hello Spark Streaming |
| 6 | Streaming with SQL |

---

## SQL operations
정적 데이터를 갖고있는 테이블에 다양한 연산을 적용하듯이 프로그래밍

In [4]:
# SparkSQL 에서 제공되는 함수를 사용해서 transformation
# lines 데이터프레임의 value 컬럼을 공백으로 split 하고, 각 원소들을 row 로 하는 word 컬럼 생성
words = lines.select(explode(split(lines.value, " ")).alias("word"))
print(words)

DataFrame[word: string]


|   | word |
| --- | --- |
| 1 | Hello |
| 2 | World |
| 3 | Word |
| 4 | Count |
| 5 | Spark |
| 6 | Streaming |
| 7 | Streaming |
| 8 | World |
| 9 | Hello |
| 10 | Spark |
| 11 | Streaming |
| 12 | Streaming |
| 13 | with |
| 14 | SQL |

In [5]:
# words 데이터프레임에서 word 로 집계한다음 각 원소들의 수를 센 결과 데이터프레임이 wordCounts
# 일반적인 테이블에서 집계하는 것과 동일하지만, 스파크가 지속적으로 소켓에서 데이터를 확인해서, incremental query 를 실행시킴
wordCounts = words.groupBy("word").count()
print(wordCounts)

DataFrame[word: string, count: bigint]


|   | word | count |
| --- | --- | --- |
| 1 | Hello | 2 |
| 2 | World | 2 |
| 3 | Word | 1 |
| 4 | Count | 1 |
| 5 | Spark | 2 |
| 6 | Streaming | 4 |
| 7 | with | 1 |
| 8 | SQL | 1 |

---

## 싱크 (Sink)
failure 에 대비하여 멱등하게(idempotent, 스트림 처리를 다시 실행시키더라도 똑같은 결과를 내게) 설계

In [ ]:
# 결과테이블을 complete 모드로, 콘솔 싱크에 write
query = wordCounts.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()

### outputMode
- append : 이번 인터벌에 들어온 새로운 row 들만 처리, 수신한 스트림을 처리한 결과가 수정되지 않는 경우(proj, filtering, mapping 등의 선형 변환)
- complete : 결과 테이블을 싹 갱신해서 싱크에 write. 실질적으로는 낮은 카디널리티 기준에 따라 집계하는 경우에만 권장(row 가 적어야 함)
- update : 결과테이블 중 바뀌는 row들만 write (즉, 집계 등과 같이 결과테이블의 다른 row 에 영향을 주는 스트림처리가 아니라면 append 와 동일)

### format
- console : stdout 에 출력
- memory : 메모리에 테이블을 만들고 지속적인 갱신
- file : 파일시스템에 특정 형식으로 - csv, json, avro, text 등
- kafka : 카프카 토픽에 프로듀싱

### start
DataStreamWriter(```writeStream```) 가 실제로 수행될 수 있게 전체 Job을 스트리밍 연산으로 구체화시키고 내부적으로 스케줄링 프로세스를 시작

쿼리를 관리하는 StreamingQuery 객체 반환

---